# Lecture 22
## Tuesday, November 15th, 2022
### SQL Database Exercise (I)

# `SQLite` Exercises

Today you will work with the `candidates.txt` and `contributors.txt` datasets to
create a database in `python` using `SQLite`. This is a hands-on lecture.
You may do these tasks in a Jupyter notebook.

The exercises will consist of a sequence of steps to help illustrate basic
commands.

<a id='deliverables'></a>
# Exercise Deliverables

> 1. Copy the Jupyter notebook along with `candidates.txt` and
>    `contributors.txt` into `lab/pp12` in your private Git repository and
>    commit on your default branch.
> 2. For each step in the exercise notebook, there are instructions labeled
>    "**Do the following:**" (except for Setup and Interlude). Put all the code
>    for those instructions in _code cell(s) immediately following the
>    instructions_. The code in those cells should be regular Python code.
>    You should place comments where appropriate that describe your intentions.
>    **Note:** To get the
>    `pandas` tables to display in a cell, use `display()`.
> 3. Save and close your database.  Be sure to upload your database in
>    `lab/pp12` as well Please name your database **`lecture22.sqlite`**.

## Table of Contents

[Setup](#setup)

[Interlude](#interlude): Not required but highly recommended.

[Step 1](#step_1)

[Step 2](#step_2)

[Step 3](#step_3)

[Step 4](#step_4)

[Step 5](#step_5)

[Step 6](#step_6)

[Step 7](#step_7)

[Step 8](#step_8)


<a id='setup'></a>
# Setup

You should import `sqlite3` again like in the lecture.

In [1]:
import sqlite3

We will also use a basic `pandas` feature to display tables in the database.

In [2]:
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)

from IPython.display import display

Now we create the tables in the database (similar to lecture).

In [3]:
db = sqlite3.connect('lecture22.sqlite')
cursor = db.cursor()
cursor.execute("DROP TABLE IF EXISTS candidates")
cursor.execute("DROP TABLE IF EXISTS contributors")
cursor.execute("PRAGMA foreign_keys=1")

cursor.execute('''CREATE TABLE candidates (
               id INTEGER PRIMARY KEY NOT NULL, 
               first_name TEXT, 
               last_name TEXT, 
               middle_init TEXT, 
               party TEXT NOT NULL)''')

db.commit() # Commit changes to the database

cursor.execute('''CREATE TABLE contributors (
          id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, 
          last_name TEXT, 
          first_name TEXT, 
          middle_name TEXT, 
          street_1 TEXT, 
          street_2 TEXT, 
          city TEXT, 
          state TEXT, 
          zip TEXT, 
          amount REAL, 
          date DATETIME, 
          candidate_id INTEGER NOT NULL, 
          FOREIGN KEY(candidate_id) REFERENCES candidates(id))''')

db.commit()

Next we load the data for the candidates:

In [4]:
with open ("candidates.txt") as candidates:
    next(candidates) # jump over the header
    for line in candidates.readlines():
        cid, first_name, last_name, middle_name, party = line.strip().split('|')
        vals_to_insert = (int(cid), first_name, last_name, middle_name, party)
        cursor.execute('''INSERT INTO candidates 
                  (id, first_name, last_name, middle_init, party)
                  VALUES (?, ?, ?, ?, ?)''', vals_to_insert)
db.commit()

<a id='interlude'></a>
## Interlude

Now that you have values in the tables of the database, it would be convenient
to be able to visualize those tables in some way.  We'll write a little helper
function to accomplish this.

In [5]:
def viz_tables(cols, query):
    q = cursor.execute(query).fetchall()
    framelist = dict()
    for i, col_name in enumerate(cols):
        framelist[col_name] = [row[i] for row in q]
    return pd.DataFrame.from_dict(framelist)

Here's how we can use our helper function.  It gives a pretty nice visualization
of our table.  You should do the same with the `contributors` table.

In [6]:
candidate_cols = [col[1] for col in cursor.execute("PRAGMA table_info(candidates)")]
query = '''SELECT * FROM candidates'''
viz_tables(candidate_cols, query)

,id,first_name,last_name,middle_init,party
0,16,Mike,Huckabee,,R
1,20,Barack,Obama,,D
2,22,Rudolph,Giuliani,,R
3,24,Mike,Gravel,,D
4,26,John,Edwards,,D
5,29,Bill,Richardson,,D
6,30,Duncan,Hunter,,R
7,31,Dennis,Kucinich,,D
8,32,Ron,Paul,,R
9,33,Joseph,Biden,,D


<a id='step_1'></a>
# Step 1

We still need to load the data in the contributors table.

### Do the following:

* Load the data in the `contributors` table by loading the data from the
  `contributors.txt` file.  **You are not allowed to use a `for`-loop in this
  task (comprehensions are allowed)**.
* Display the contributors table.

In [7]:
data = pd.read_csv("./contributors.txt", delimiter="|")
data.to_sql("contributors", db, if_exists="replace")

contributor_cols = [col[1] for col in cursor.execute("PRAGMA table_info(contributors)")]
query = '''SELECT * FROM contributors'''
viz_tables(contributor_cols, query)

,index,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,0,None,Agee,Steven,None,549 Laurel Branch Road,None,Floyd,VA,24091,500.00,2007-06-30,16
1,1,None,Ahrens,Don,None,4034 Rennellwood Way,None,Pleasanton,CA,94566,250.00,2007-05-16,16
2,2,None,Ahrens,Don,None,4034 Rennellwood Way,None,Pleasanton,CA,94566,50.00,2007-06-18,16
3,3,None,Ahrens,Don,None,4034 Rennellwood Way,None,Pleasanton,CA,94566,100.00,2007-06-21,16
4,4,None,Akin,Charles,None,10187 Sugar Creek Road,None,Bentonville,AR,72712,100.00,2007-06-16,16
5,5,None,Akin,Mike,None,181 Baywood Lane,None,Monticello,AR,71655,1500.00,2007-05-18,16
6,6,None,Akin,Rebecca,None,181 Baywood Lane,None,Monticello,AR,71655,500.00,2007-05-18,16
7,7,None,Aldridge,Brittni,None,"808 Capitol Square Place, SW",None,Washington,DC,20024,250.00,2007-06-06,16
8,8,None,Allen,John D.,None,1052 Cannon Mill Drive,None,North Augusta,SC,29860,1000.00,2007-06-11,16
9,9,None,Allen,John D.,None,1052 Cannon Mill Drive,None,North Augusta,SC,29860,1300.00,2007-06-29,16


<a id='step_2'></a>
# Step 2:  Various Queries

We can query our database for entries with certain characteristics.  For
example, we can query the `candidates` table for entries whose middle name
fields are not empty.

In [8]:
query = '''SELECT * FROM candidates WHERE middle_init <> ""'''
viz_tables(candidate_cols, query)

,id,first_name,last_name,middle_init,party
0,34,Hillary,Clinton,R.,D
1,39,Christopher,Dodd,J.,D
2,41,Fred,Thompson,D.,R


We can also see how many entries satisfy the query:

In [9]:
print("{} candidates have a middle initial.".format(viz_tables(candidate_cols, query).shape[0]))

3 candidates have a middle initial.


This used the `shape` method on the returned `pandas` table. You'll get to
practice counting in SQL later.

### Do the following queries:

* Display the contributors where the state is "PA"
* Display the contributors where the amount contributed is greater than
  $\$1000.00$.
* Display the contributors from the state "UT" where the amount contributed is
  greater than $\$1000.00$.
* Display the contributors who didn't list their state
  - **Hint**:  Match `state` to the empty string
* Display the contributors from "WA" or "PA"
* Display the contributors who contributed between $\$100.00$ and $\$200.00$.
  - **Hint**: You can use the `BETWEEN 100.00 and 200.00` clause.

In [12]:
query =  " SELECT * FROM contributors WHERE state ='PA' "
viz_tables(contributor_cols, query)

,index,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,70,None,BUCKLEY,WALTER,W.,1635 COUNTRY ROAD,None,BETHLEHEM,PA,180155718,-100.0,2008-03-05,22
1,71,None,BUCKLEY,MARJORIE,B.,1635 COUNTRY ROAD,None,BETHLEHEM,PA,180155718,-100.0,2008-03-05,22
2,93,None,Raught,Philip,M,4714 Plum Way,None,Pittsburgh,PA,15201,-1046.0,2008-04-21,32
3,94,None,Ferrara,Judith,D,1508 Waterford Road,None,Yardley,PA,19067,-1100.0,2008-04-21,32
4,165,None,ABEL,JOHN,H.,422 THOMAS STREET,None,BETHLEHEM,PA,180153316,200.0,2008-01-22,37


In [13]:
query =  " SELECT * FROM contributors WHERE amount >1000 "
viz_tables(contributor_cols, query)

,index,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,5,None,Akin,Mike,None,181 Baywood Lane,None,Monticello,AR,71655,1500.0,2007-05-18,16
1,9,None,Allen,John D.,None,1052 Cannon Mill Drive,None,North Augusta,SC,29860,1300.0,2007-06-29,16
2,13,None,Altes,R.D.,None,8600 Moody Road,None,Fort Smith,AR,72903,2300.0,2007-06-21,16
3,15,None,Anthony,John,None,211 Long Island Drive,None,Hot Springs,AR,71913,2300.0,2007-06-12,16
4,21,None,Baker,David,None,2550 Adamsbrooke Drive,None,Conway,AR,72034,2300.0,2007-04-11,16
5,28,None,Buckel,Linda,None,PO Box 683130,None,Park City,UT,840683130,2300.0,2007-08-14,20
6,30,None,Buckel,Linda,None,PO Box 683130,None,Park City,UT,840683130,4600.0,2007-08-14,20
7,33,None,Buck,Blaine,M,45 Eaton Ave,None,Camden,ME,48431752,2300.0,2007-09-30,20
8,45,None,Buchanan,John,None,2025 NW 29th Rd,None,Boca Raton,FL,334316303,1300.0,2007-08-09,20
9,135,None,ABRAMOWITZ,NIRA,None,411 HARBOR ROAD,None,SOUTHPORT,CT,68901376,2300.0,2007-09-14,35


In [14]:
query =  " SELECT * FROM contributors WHERE state = 'UT' AND amount >1000 "
viz_tables(contributor_cols, query)

,index,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,28,None,Buckel,Linda,None,PO Box 683130,None,Park City,UT,840683130,2300.0,2007-08-14,20
1,30,None,Buckel,Linda,None,PO Box 683130,None,Park City,UT,840683130,4600.0,2007-08-14,20


In [16]:
query =  " SELECT * FROM contributors WHERE state IS NULL or state = '' "
viz_tables(contributor_cols, query)

,index,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,125,None,BOURNE,TRAVIS,None,LAGE KAART 77,None,BRASSCHATT,None,2930,-500.0,2008-11-20,35


In [17]:
query =  " SELECT * FROM contributors WHERE state IN('WA','PA') "
viz_tables(contributor_cols, query)

,index,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,62,None,BURKE,SUZANNE,M.,3401 EVANSTON,None,SEATTLE,WA,981038677,-700.0,2008-03-05,22
1,70,None,BUCKLEY,WALTER,W.,1635 COUNTRY ROAD,None,BETHLEHEM,PA,180155718,-100.0,2008-03-05,22
2,71,None,BUCKLEY,MARJORIE,B.,1635 COUNTRY ROAD,None,BETHLEHEM,PA,180155718,-100.0,2008-03-05,22
3,93,None,Raught,Philip,M,4714 Plum Way,None,Pittsburgh,PA,15201,-1046.0,2008-04-21,32
4,94,None,Ferrara,Judith,D,1508 Waterford Road,None,Yardley,PA,19067,-1100.0,2008-04-21,32
5,100,None,Aaronson,Rebecca,None,2000 Village Green Dr Apt 12,None,Mill Creek,WA,980125787,100.0,2008-02-08,34
6,106,None,Aaronson,Rebecca,None,2000 Village Green Dr Apt 12,None,Mill Creek,WA,980125787,100.0,2008-02-14,34
7,165,None,ABEL,JOHN,H.,422 THOMAS STREET,None,BETHLEHEM,PA,180153316,200.0,2008-01-22,37


In [18]:
query =  " SELECT * FROM contributors WHERE amount between 100 AND 200 "
viz_tables(contributor_cols, query)

,index,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,3,None,Ahrens,Don,None,4034 Rennellwood Way,None,Pleasanton,CA,94566,100.0,2007-06-21,16
1,4,None,Akin,Charles,None,10187 Sugar Creek Road,None,Bentonville,AR,72712,100.0,2007-06-16,16
2,12,None,Allison,Rebecca,None,3206 Summit Court,None,Little Rock,AR,72227,200.0,2007-06-12,16
3,17,None,Arbogast,Robert,None,12900 State Route 56 SE,None,Mount Sterling,OH,43143,100.0,2007-06-22,16
4,27,None,Buckheit,Bruce,None,8904 KAREN DR,None,FAIRFAX,VA,220312731,100.0,2007-09-19,20
5,31,None,Buck,Thomas,None,4206 Terrace Street,None,Kansas City,MO,64111,100.0,2007-09-25,20
6,32,None,Buck,Jay,K.,1855 Old Willow Rd Unit 322,None,Northfield,IL,600932918,200.0,2007-09-12,20
7,37,None,Bucher,Ida,M,1400 Warnall Ave,None,Los Angeles,CA,900245333,100.0,2007-07-10,20
8,46,None,Buchanan,John,None,2025 NW 29th Rd,None,Boca Raton,FL,334316303,200.0,2007-08-14,20
9,100,None,Aaronson,Rebecca,None,2000 Village Green Dr Apt 12,None,Mill Creek,WA,980125787,100.0,2008-02-08,34


<a id='step_3'></a>
# Step 3:  Sorting

It could be beneficial to sort by one of the attributes in the database.  The
following cell contains a basic sorting demo. Run it and try to understand what
happened.

In [19]:
query = '''SELECT * FROM candidates ORDER BY id DESC'''
viz_tables(candidate_cols, query)

,id,first_name,last_name,middle_init,party
0,41,Fred,Thompson,D.,R
1,39,Christopher,Dodd,J.,D
2,38,Tom,Tancredo,,R
3,37,John,McCain,,R
4,36,Samuel,Brownback,,R
5,35,Mitt,Romney,,R
6,34,Hillary,Clinton,R.,D
7,33,Joseph,Biden,,D
8,32,Ron,Paul,,R
9,31,Dennis,Kucinich,,D


### Do the following sorts on the `contributors` table:

* Sort the `candidates` table by `last_name`.
* Sort the `contributors` table by the `amount` in descending order where `amount`
  is restricted to be between $\$1000.00$ and $\$5000.00$.
  - **Hint:** In your SQL command, start with getting the amount between the
    specified range followed by the sort. This will all be done in one line.
* Sort the contributors who donated between $\$1000.00$ and $\$5000.00$ by
  `candidate_id` and then by `amount` in descending order.
  - **Hint**:  Multiple orderings can be accomplished by separating requests
    after `ORDER BY` with commas.
  - e.g. `ORDER BY amount ASC, last_name DESC`

In [20]:
query = '''SELECT * FROM candidates ORDER BY last_name DESC'''
viz_tables(candidate_cols, query)

,id,first_name,last_name,middle_init,party
0,41,Fred,Thompson,D.,R
1,38,Tom,Tancredo,,R
2,35,Mitt,Romney,,R
3,29,Bill,Richardson,,D
4,32,Ron,Paul,,R
5,20,Barack,Obama,,D
6,37,John,McCain,,R
7,31,Dennis,Kucinich,,D
8,30,Duncan,Hunter,,R
9,16,Mike,Huckabee,,R


In [21]:
query = '''SELECT * FROM contributors WHERE amount BETWEEN 1000 AND 5000 ORDER BY amount DESC'''
viz_tables(candidate_cols, query)

,id,first_name,last_name,middle_init,party
0,30,None,Buckel,Linda,None
1,159,None,ABATE,MARIA,ELENA
2,13,None,Altes,R.D.,None
3,15,None,Anthony,John,None
4,21,None,Baker,David,None
5,28,None,Buckel,Linda,None
6,33,None,Buck,Blaine,M
7,135,None,ABRAMOWITZ,NIRA,None
8,5,None,Akin,Mike,None
9,9,None,Allen,John D.,None


In [22]:
query = '''SELECT * FROM contributors WHERE amount BETWEEN 1000 AND 5000 ORDER BY candidate_id , amount DESC'''
viz_tables(candidate_cols, query)

,id,first_name,last_name,middle_init,party
0,13,None,Altes,R.D.,None
1,15,None,Anthony,John,None
2,21,None,Baker,David,None
3,5,None,Akin,Mike,None
4,9,None,Allen,John D.,None
5,8,None,Allen,John D.,None
6,10,None,Allison,John W.,None
7,11,None,Allison,Rebecca,None
8,19,None,Atiq,Omar,None
9,20,None,Atiq,Omar,None


<a id='step_4'></a>
# Step 4:  Selecting Columns

So far, we've been selecting all columns from a table (i.e. `SELECT * FROM`).
Often, we just want to select specific columns (e.g. `SELECT amount FROM`).

In [23]:
query = '''SELECT last_name, party FROM candidates'''
viz_tables(['last_name', 'party'], query)

,last_name,party
0,Huckabee,R
1,Obama,D
2,Giuliani,R
3,Gravel,D
4,Edwards,D
5,Richardson,D
6,Hunter,R
7,Kucinich,D
8,Paul,R
9,Biden,D


Using the `DISTINCT` clause, you remove duplicate rows.

In [24]:
query = '''SELECT DISTINCT party FROM candidates'''
viz_tables(['party'], query)

,party
0,R
1,D


### Do the following:

* Get the first and last name of contributors.  Make sure each row has distinct
  values.

In [28]:
query = '''SELECT  DISTINCT lower(first_name), lower(last_name) FROM contributors'''
viz_tables(['first_name', 'last_name'], query)

,first_name,last_name
0,steven,agee
1,don,ahrens
2,charles,akin
3,mike,akin
4,rebecca,akin
5,brittni,aldridge
6,john d.,allen
7,john w.,allison
8,rebecca,allison
9,r.d.,altes


<a id='step_5'></a>
# Step 5:  Altering Tables

The `ALTER` clause allows us to modify tables in our database.  Here, we add a
new column to our candidates table called `full_name`.

In [29]:
cursor.execute('''ALTER TABLE candidates ADD COLUMN full_name TEXT''')
candidate_cols = [col[1] for col in cursor.execute("PRAGMA table_info(candidates)")]
viz_tables(candidate_cols, '''SELECT * FROM candidates''')

,id,first_name,last_name,middle_init,party,full_name
0,16,Mike,Huckabee,,R,None
1,20,Barack,Obama,,D,None
2,22,Rudolph,Giuliani,,R,None
3,24,Mike,Gravel,,D,None
4,26,John,Edwards,,D,None
5,29,Bill,Richardson,,D,None
6,30,Duncan,Hunter,,R,None
7,31,Dennis,Kucinich,,D,None
8,32,Ron,Paul,,R,None
9,33,Joseph,Biden,,D,None


What if we want to rename or delete a column?  It can't be done with `SQLite`
with a single command.  We need to follow some roundabout steps (see [`SQLite`
ALTER TABLE](http://www.sqlitetutorial.net/sqlite-alter-table/)).  We won't
consider this case at the moment.

For now, let's put a few commands together to populate the `full_name` column.

In [30]:
candidate_cols = [col[1] for col in cursor.execute("PRAGMA table_info(candidates)")] # regenerate columns with full_name
query = '''SELECT id, last_name, first_name FROM candidates''' # Select a few columns
full_name_and_id = [(attr[1] + ", " + attr[2], attr[0]) for attr in cursor.execute(query).fetchall()] # List of tuples: (full_name, id)

update = '''UPDATE candidates SET full_name = ? WHERE id = ?''' # Update the table
for rows in full_name_and_id:
    cursor.execute(update, rows)

query = '''SELECT * FROM candidates'''
viz_tables(candidate_cols, query)

,id,first_name,last_name,middle_init,party,full_name
0,16,Mike,Huckabee,,R,"Huckabee, Mike"
1,20,Barack,Obama,,D,"Obama, Barack"
2,22,Rudolph,Giuliani,,R,"Giuliani, Rudolph"
3,24,Mike,Gravel,,D,"Gravel, Mike"
4,26,John,Edwards,,D,"Edwards, John"
5,29,Bill,Richardson,,D,"Richardson, Bill"
6,30,Duncan,Hunter,,R,"Hunter, Duncan"
7,31,Dennis,Kucinich,,D,"Kucinich, Dennis"
8,32,Ron,Paul,,R,"Paul, Ron"
9,33,Joseph,Biden,,D,"Biden, Joseph"


Here's another update, this time on an existing column.

In [31]:
update = '''UPDATE candidates SET full_name = "WINNER" WHERE last_name = "Obama"'''
cursor.execute(update)
update = '''UPDATE candidates SET full_name = "RUNNER-UP" WHERE last_name = "McCain"'''
cursor.execute(update)
viz_tables(candidate_cols, query)

,id,first_name,last_name,middle_init,party,full_name
0,16,Mike,Huckabee,,R,"Huckabee, Mike"
1,20,Barack,Obama,,D,WINNER
2,22,Rudolph,Giuliani,,R,"Giuliani, Rudolph"
3,24,Mike,Gravel,,D,"Gravel, Mike"
4,26,John,Edwards,,D,"Edwards, John"
5,29,Bill,Richardson,,D,"Richardson, Bill"
6,30,Duncan,Hunter,,R,"Hunter, Duncan"
7,31,Dennis,Kucinich,,D,"Kucinich, Dennis"
8,32,Ron,Paul,,R,"Paul, Ron"
9,33,Joseph,Biden,,D,"Biden, Joseph"


### Do the following:

* Add a new column to the contributors table called `full_name`.  The value in
  that column should be in the form `last_name, first_name`.
* Change the value in the `full_name` column to the string `"Too Much"` if
  someone donated more than $\$1000.00$.

<a id='step_6'></a>
# Step 6: Aggregation

You can perform reduction operations on the values in the database.  For
example, you can compute the maximum, minimum, sum or the total number from
multiple input values. Here's a little example:

In [32]:
contributor_cols = [col[1] for col in cursor.execute("PRAGMA table_info(contributors)")] # You've already done this part in your solution.
function = '''SELECT *, MAX(amount) AS max_amount FROM contributors'''
viz_tables(contributor_cols, function)

,index,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,30,None,Buckel,Linda,None,PO Box 683130,None,Park City,UT,840683130,4600.0,2007-08-14,20


### Do the following:

* Modify the demo to only output the max amount.
    * **Hints:**
        - Instead of using `SELECT *, MAX(amount) ...` you can try `SELECT
          MAX(amount) ...`
        - You will want to use `cursor.execute()` here and then `fetchall()`.
        - Do not display your results in a table. It is sufficient to write
          the answer out to the screen as a single number.
* Count how many donations there were above $\$1000.00$.
    * **Hint:** There is a `COUNT` function.
* Calculate the average *positive* donation.
    * **Hint:** There is an `AVG` function.
* Calculate the average contribution from each state and display in a table.
  Restrict to positive values again.
  - **Hint**:  Use code that looks like: `"SELECT state,SUM(amount) FROM
    contributors GROUP BY state"`.

<a id='step_7'></a>
# Step 7: DELETE

We have already noted that `SQLite` can't drop columns in a straightforward
manner.  However, it can delete rows quite simply.  Here's the syntax:

In [34]:
function = '''SELECT MAX(amount) AS max_amount FROM contributors'''
viz_tables(["max_amount"], function)

,max_amount
0,4600.0


In [35]:
function = '''SELECT count(*) AS count FROM contributors WHERE amount > 1000'''
viz_tables(["max_amount"], function)

,max_amount
0,12


In [36]:
function = '''SELECT avg(amount) AS avg_amount FROM contributors WHERE amount > 0'''
viz_tables(["avg_amount"], function)

,avg_amount
0,480.691368


In [39]:
function = '''SELECT state, avg(amount) AS avg_amount FROM contributors GROUP BY state'''
viz_tables(["state","avg_amount"], function)

,state,avg_amount
0,None,-500.000000
1,AK,403.333333
2,AR,1183.333333
3,AZ,120.000000
4,CA,-217.988261
5,CO,-1455.750000
6,CT,2300.000000
7,DC,-309.982000
8,FL,-135.000000
9,IA,250.000000


In [ ]:
deletion = '''DELETE FROM table_name WHERE condition'''

In [43]:
function = '''DELETE FROM contributors WHERE last_name = "Ahrens"'''
viz_tables(["state","avg_amount"], function)

,state,avg_amount


### Do the following:

* Delete rows in the `contributors` table with last name "Ahrens".

<a id='step_8'></a>
# Step 8:  LIMIT

The `LIMIT` clause offers convenient functionality.  It allows you to constrain
the number of rows returned by your query.  It shows up in many guises.

In [44]:
query = '''SELECT * FROM candidates LIMIT 3'''
viz_tables(candidate_cols, query)

,id,first_name,last_name,middle_init,party,full_name
0,16,Mike,Huckabee,,R,"Huckabee, Mike"
1,20,Barack,Obama,,D,WINNER
2,22,Rudolph,Giuliani,,R,"Giuliani, Rudolph"


In [45]:
query = '''SELECT * FROM candidates LIMIT 4 OFFSET 5'''
viz_tables(candidate_cols, query)

,id,first_name,last_name,middle_init,party,full_name
0,29,Bill,Richardson,,D,"Richardson, Bill"
1,30,Duncan,Hunter,,R,"Hunter, Duncan"
2,31,Dennis,Kucinich,,D,"Kucinich, Dennis"
3,32,Ron,Paul,,R,"Paul, Ron"


In [46]:
query = '''SELECT * FROM candidates ORDER BY last_name LIMIT 4 OFFSET 5'''
viz_tables(candidate_cols, query)

,id,first_name,last_name,middle_init,party,full_name
0,22,Rudolph,Giuliani,,R,"Giuliani, Rudolph"
1,24,Mike,Gravel,,D,"Gravel, Mike"
2,16,Mike,Huckabee,,R,"Huckabee, Mike"
3,30,Duncan,Hunter,,R,"Hunter, Duncan"


### Do the following:

* Query and display the ten most generous donors.
* Query and display the ten least generous donors who donated a positive amount
  of money (since the data we have has some negative numbers in it...).

In [51]:
query = '''SELECT * FROM contributors ORDER BY amount DESC LIMIT 10'''
viz_tables(contributor_cols, query)

,index,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,30,None,Buckel,Linda,None,PO Box 683130,None,Park City,UT,840683130,4600.0,2007-08-14,20
1,159,None,ABATE,MARIA,ELENA,1291 NIGHTINGALE AVENUE,None,MIAMI SPRINGS,FL,331663832,2600.0,2008-01-25,37
2,13,None,Altes,R.D.,None,8600 Moody Road,None,Fort Smith,AR,72903,2300.0,2007-06-21,16
3,15,None,Anthony,John,None,211 Long Island Drive,None,Hot Springs,AR,71913,2300.0,2007-06-12,16
4,21,None,Baker,David,None,2550 Adamsbrooke Drive,None,Conway,AR,72034,2300.0,2007-04-11,16
5,28,None,Buckel,Linda,None,PO Box 683130,None,Park City,UT,840683130,2300.0,2007-08-14,20
6,33,None,Buck,Blaine,M,45 Eaton Ave,None,Camden,ME,48431752,2300.0,2007-09-30,20
7,135,None,ABRAMOWITZ,NIRA,None,411 HARBOR ROAD,None,SOUTHPORT,CT,68901376,2300.0,2007-09-14,35
8,5,None,Akin,Mike,None,181 Baywood Lane,None,Monticello,AR,71655,1500.0,2007-05-18,16
9,9,None,Allen,John D.,None,1052 Cannon Mill Drive,None,North Augusta,SC,29860,1300.0,2007-06-29,16


In [50]:
query = '''SELECT * FROM contributors WHERE amount > 0  ORDER BY amount ASC LIMIT 10'''
viz_tables(contributor_cols, query)

,index,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,26,None,Buckler,Steve,None,24351 Armada Dr,None,Dana Point,CA,926291306,25.0,2007-08-16,20
1,49,None,Harrison,Ryan,None,2247 3rd St,None,La Verne,CA,917504918,25.0,2007-07-26,20
2,101,None,Aarons,Elaine,None,481 Buck Island Rd Apt 17A,APT 17A,West Yarmouth,MA,26733300,25.0,2008-02-26,34
3,140,None,ABEGG,PATRICIA,T.,1862 E. 5150 S.,None,SALT LAKE CITY,UT,841176911,25.0,2007-09-17,35
4,143,None,ABEGG,PATRICIA,T.,1862 E. 5150 S.,None,SALT LAKE CITY,UT,841176911,25.0,2007-08-06,35
5,144,None,ABEGG,PATRICIA,T.,1862 E. 5150 S.,None,SALT LAKE CITY,UT,841176911,25.0,2007-07-10,35
6,160,None,ABAIR,PETER,None,40 EVANS STREET,None,WATERTOWN,MA,24722150,25.0,2008-01-09,37
7,158,None,ABBO,PAULINE,MORENCY,10720 JACOB LANE,None,WHITE LAKE,MI,483862274,35.0,2008-01-07,37
8,18,None,Ardle,William,None,412 Dakota Avenue,None,Springfield,OH,45504,50.0,2007-06-28,16
9,25,None,Buckler,Steve,None,24351 Armada Dr,None,Dana Point,CA,926291306,50.0,2007-07-30,20


# Save

Don't forget to save all of these changes to your database using `db.commit()`.
Before closing your editor or IDE, be sure to close the database connection with
`db.close()`.

In [52]:
db.commit()

In [54]:
db.close()